In [1]:
import pandas as pd
import numpy as np

In [2]:
revenue = pd.read_excel('/Volumes/External/OneDrive - Emory University/MSBA/Spring/ISOM 689 - Capstone/data/DLY_HTL_COMPSET_PERF_MASKED.xlsx')
competitors = pd.read_excel('/Volumes/External/OneDrive - Emory University/MSBA/Spring/ISOM 689 - Capstone/data/RATE_SHOP_COMPET_HTL_masked.xlsx')

In [3]:
revenue.head()

,FAC_ID,RPT_DAY_DT_TY,htl_rms_avl,htl_rms_act,htl_rev_const_usd_act,comp_rms_avl,comp_rms_act,comp_rev_const_usd_act,htl_occ,htl_adr,htl_revpar,comp_occ,comp_adr,comp_revpar,mpi,ari,rgi
0,3,2017-01-01,122.0,23.0,2295.0,122.0,18.633441,1673.086817,0.188525,99.782609,18.811475,0.152733,89.789471,13.713826,1.234341,1.111295,1.371716
1,3,2017-01-02,122.0,23.0,2378.0,122.0,19.418006,1888.254019,0.188525,103.391304,19.491803,0.159164,97.242426,15.477492,1.184468,1.063232,1.259364
2,3,2017-01-03,122.0,30.0,3104.0,122.0,41.974277,4078.173633,0.245902,103.466667,25.442623,0.344051,97.158877,33.427653,0.714724,1.064922,0.761125
3,3,2017-01-04,122.0,46.0,4664.0,122.0,54.919614,5540.408360,0.377049,101.391304,38.229508,0.450161,100.882143,45.413183,0.837587,1.005047,0.841815
4,3,2017-01-05,122.0,45.0,4676.0,122.0,46.681672,4449.077170,0.368852,103.911111,38.327869,0.382637,95.306723,36.467846,0.963975,1.090281,1.051004


In [4]:
revenue_t = revenue.drop(columns=['htl_rms_act', 'comp_rms_act', 'comp_rev_const_usd_act', 'htl_occ', 'htl_adr', 'htl_revpar', 'comp_occ', 'comp_adr', 'comp_revpar', 'comp_revpar'])

In [5]:
revenue['week'] = revenue['RPT_DAY_DT_TY'].dt.week
revenue['weekday'] = revenue['RPT_DAY_DT_TY'].dt.weekday
revenue['weekday'] = revenue['weekday'].apply(lambda x: 1 if x>=5 else 0)

In [6]:
revenue_t = revenue.groupby(['RPT_DAY_DT_TY']).agg({
    'htl_rms_avl':'sum',
    'htl_rms_act':'sum',
    'htl_rev_const_usd_act':'sum',
    'comp_rms_avl': 'sum',
    'comp_rms_act':'sum',
    'htl_occ':'mean',
    'htl_adr':'mean',
    'htl_revpar':'mean',
    'comp_occ':'mean',
    'comp_adr':'mean',
    'comp_revpar':'mean',
    'mpi':'mean',
    'ari':'mean',
    'rgi':'mean'
}).reset_index()

In [7]:
revenue_t.head()

,RPT_DAY_DT_TY,htl_rms_avl,htl_rms_act,htl_rev_const_usd_act,comp_rms_avl,comp_rms_act,htl_occ,htl_adr,htl_revpar,comp_occ,comp_adr,comp_revpar,mpi,ari,rgi
0,2017-01-01,44584.0,18069.0,1919278.0,44584.0,18402.412654,0.388624,98.868890,40.253241,0.397860,91.176314,38.429649,1.016219,1.098248,1.101329
1,2017-01-02,44506.0,16156.0,1644727.0,44506.0,16145.604331,0.357318,98.323008,35.973401,0.353072,90.219076,32.891860,1.054211,1.104781,1.153845
2,2017-01-03,44506.0,21533.0,2190306.0,44506.0,20907.894596,0.485761,99.346875,48.983506,0.463943,93.271486,43.972421,1.070747,1.075261,1.153374
3,2017-01-04,44506.0,23574.0,2411719.0,44506.0,23034.766894,0.531713,99.995536,53.912270,0.511141,93.920616,48.676046,1.061534,1.074512,1.146913
4,2017-01-05,44506.0,22707.0,2316549.0,44506.0,22419.653502,0.507666,99.229336,51.295744,0.496153,92.750095,46.818849,1.049533,1.078654,1.136998


In [8]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from matplotlib import pyplot as plt

plot_pacf(revenue_t.htl_rev_const_usd_act)
plt.show()

ImportError: cannot import name 'datetools'

In [9]:
revenue_60 = revenue_t[['RPT_DAY_DT_TY', 'htl_rms_act']]

In [10]:
revenue_60.head()

,RPT_DAY_DT_TY,htl_rms_act
0,2017-01-01,18069.0
1,2017-01-02,16156.0
2,2017-01-03,21533.0
3,2017-01-04,23574.0
4,2017-01-05,22707.0


In [11]:
rng = list(range(0, 60)) + list(range(150, len(revenue_60)))

revenue_60 = revenue_60.iloc[rng, :]

In [12]:
for n in range(1,61):
    col_name = 'day' + str(n)
    revenue_60[col_name] = revenue_60['htl_rms_act'].shift(n)

In [13]:
revenue_60.head()

,RPT_DAY_DT_TY,htl_rms_act,day1,day2,day3,day4,day5,day6,day7,day8,...,day51,day52,day53,day54,day55,day56,day57,day58,day59,day60
0,2017-01-01,18069.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-02,16156.0,18069.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-03,21533.0,16156.0,18069.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-04,23574.0,21533.0,16156.0,18069.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-05,22707.0,23574.0,21533.0,16156.0,18069.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df = revenue_60.dropna()

In [15]:
df.head()

,RPT_DAY_DT_TY,htl_rms_act,day1,day2,day3,day4,day5,day6,day7,day8,...,day51,day52,day53,day54,day55,day56,day57,day58,day59,day60
150,2017-05-31,30323.0,34710.0,34482.0,30668.0,21466.0,33637.0,32948.0,32884.0,35905.0,...,32786.0,29142.0,19864.0,23750.0,23103.0,22707.0,23574.0,21533.0,16156.0,18069.0
151,2017-06-01,30318.0,30323.0,34710.0,34482.0,30668.0,21466.0,33637.0,32948.0,32884.0,...,32722.0,32786.0,29142.0,19864.0,23750.0,23103.0,22707.0,23574.0,21533.0,16156.0
152,2017-06-02,33259.0,30318.0,30323.0,34710.0,34482.0,30668.0,21466.0,33637.0,32948.0,...,27986.0,32722.0,32786.0,29142.0,19864.0,23750.0,23103.0,22707.0,23574.0,21533.0
153,2017-06-03,35800.0,33259.0,30318.0,30323.0,34710.0,34482.0,30668.0,21466.0,33637.0,...,26768.0,27986.0,32722.0,32786.0,29142.0,19864.0,23750.0,23103.0,22707.0,23574.0
154,2017-06-04,25536.0,35800.0,33259.0,30318.0,30323.0,34710.0,34482.0,30668.0,21466.0,...,28559.0,26768.0,27986.0,32722.0,32786.0,29142.0,19864.0,23750.0,23103.0,22707.0


In [16]:
X_train = df[df['RPT_DAY_DT_TY'] < '2018-07-01'].drop(columns=['RPT_DAY_DT_TY'])
X_test = df[df['RPT_DAY_DT_TY'] >= '2018-07-01'].drop(columns=['RPT_DAY_DT_TY'])
y_train = df[df['RPT_DAY_DT_TY'] < '2018-07-01']['htl_rms_act']
y_test = df[df['RPT_DAY_DT_TY'] >= '2018-07-01']['htl_rms_act']

# Regression Tree

In [17]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

gs_dt = GridSearchCV(estimator=DecisionTreeRegressor(random_state=41),
                  param_grid=[{'max_depth': [1, 2, 3, 4, 5, 6, 7, None],}],
                  cv=5,
                  n_jobs=-1)

gs_dt.fit(X_train, y_train)

print(gs_dt.best_score_)
print(gs_dt.best_params_)
print(gs_dt.best_estimator_)

0.9964333911695011
{'max_depth': 6}
DecisionTreeRegressor(criterion='mse', max_depth=6, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=41, splitter='best')


In [19]:
gs_dt.get_params()

{'cv': 5,
 'error_score': 'raise',
 'estimator': DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=41, splitter='best'),
 'estimator__criterion': 'mse',
 'estimator__max_depth': None,
 'estimator__max_features': None,
 'estimator__max_leaf_nodes': None,
 'estimator__min_impurity_decrease': 0.0,
 'estimator__min_impurity_split': None,
 'estimator__min_samples_leaf': 1,
 'estimator__min_samples_split': 2,
 'estimator__min_weight_fraction_leaf': 0.0,
 'estimator__presort': False,
 'estimator__random_state': 41,
 'estimator__splitter': 'best',
 'fit_params': None,
 'iid': True,
 'n_jobs': -1,
 'param_grid': [{'max_depth': [1, 2, 3, 4, 5, 6, 7, None]}],
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': 'warn',
 'scoring': None,
 'ver

In [20]:
y_pred = gs_dt.predict(X_test)

In [21]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(y_test, y_pred))
rms

247.5370922095846

In [22]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true))

In [23]:
mean_absolute_percentage_error(np.array(y_test), y_pred)

0.005522159741633205

# Linear Regression

In [24]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()

reg.fit(X_train, y_train)
reg.score(X_train, y_train)

1.0

In [25]:
y_pred_linear = reg.predict(np.array(X_test))

In [26]:
mean_absolute_percentage_error(y_test, y_pred_linear)

4.610356120355254e-16

In [27]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_test, y_pred_linear)

3.623621552370229e-22

In [35]:
y_pred

array([25275.        , 26354.7       , 22964.75      , 23372.        ,
       30315.2       , 36051.        , 37613.        , 27811.        ,
       38125.42857143, 41928.        , 41928.        , 39013.5       ,
       39408.5       , 41928.        , 29988.5       , 38719.83333333,
       41209.75      , 41209.75      , 39408.5       , 40619.        ,
       41209.75      , 29761.        , 39013.5       , 41209.75      ,
       41209.75      , 39013.5       , 40238.83333333, 41209.75      ,
       27811.        , 34606.61538462, 37613.        , 38385.85714286,
       36534.38461538, 38385.85714286, 39776.71428571, 27811.        ,
       35499.45454545, 38719.83333333, 38385.85714286, 36051.        ,
       37041.08333333, 39013.5       , 26354.7       , 35104.72727273,
       38125.42857143, 37613.        , 33842.75      , 33842.75      ,
       36534.38461538, 25275.        , 34606.61538462, 38125.42857143,
       37821.        , 33242.41666667, 32191.        , 33842.75      ,
      

In [34]:
np.array(y_test)

array([25535., 26108., 22833., 23452., 30210., 35999., 37599., 27894.,
       38006., 41752., 41822., 38956., 39482., 40975., 30207., 38723.,
       41995., 41575., 39538., 40594., 41733., 29884., 38934., 41753.,
       41101., 38995., 40195., 41505., 28021., 34740., 37725., 38542.,
       36440., 38354., 39902., 27794., 35570., 38841., 38445., 36079.,
       37048., 38913., 26432., 35003., 38131., 37620., 33969., 34013.,
       36659., 25010., 34450., 38117., 37839., 33327., 31974., 34001.,
       23026., 31681., 35168., 33779., 29706., 33558., 40534., 30096.,
       17770., 28061., 31174., 31036., 31434., 32961., 24540., 35145.,
       39298., 39144., 35414., 34688., 36054., 25634., 34605., 38687.,
       38708., 35173., 35932., 38626., 26712., 35894., 39910., 39452.,
       35465., 35249., 36767., 24344., 32646., 36550., 37040., 35035.,
       37490., 40574., 28900., 34404., 38337., 38660., 36150., 38312.,
       40268., 27851., 36558., 40642., 40762., 37803., 38529., 40376.,
      

# KNN

In [29]:
from sklearn.neighbors import KNeighborsRegressor


knn = GridSearchCV(estimator=KNeighborsRegressor(),
                  param_grid=[{'n_neighbors': [1, 3, 5, 7, 9],}],
                  cv=5,
                  n_jobs=-1)

knn.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'n_neighbors': [1, 3, 5, 7, 9]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [30]:
y_pred_knn = knn.predict(X_test)

In [31]:
mean_absolute_percentage_error(y_test, y_pred_knn)

0.09208147993112537

In [32]:
mean_squared_error(y_test, y_pred_knn)

12965086.635893695

In [2]:
import pyspark
from pyspark import SparkContext

SparkContext.getOrCreate()

<SparkContext master=local[*] appName=pyspark-shell>